In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from chickenDiseaseClassification.constants import *
from chickenDiseaseClassification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    

In [8]:
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        """Create TensorBoard callback with timestamped log directory."""
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        """Create ModelCheckpoint callback to save the best model."""
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        """Return both TensorBoard and Checkpoint callbacks as a list."""
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [10]:
import os
import time
import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request

In [11]:
tf.config.run_functions_eagerly(True)
print("Eager Execution Enabled:", tf.executing_eagerly())

class Training:
    def __init__(self, config):
        self.config = config
        self.model = None

    def get_base_model(self):
        """Load the updated base model and recompile it"""
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        print("Model recompiled successfully with new optimizer.")


    def train_valid_generator(self):
        """Prepare training and validation data generators"""
        datagenerator_kwargs = dict(rescale=1.0 / 255, validation_split=0.20)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Save the trained model"""
        model.save(path)

    def train(self, callback_list=None):
        """Train the model with callbacks and save it"""

        self.steps_per_epoch = (
            self.train_generator.samples // self.train_generator.batch_size
        )
        self.validation_steps = (
            self.valid_generator.samples // self.valid_generator.batch_size
        )

        history = self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list,
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)
        return history

Eager Execution Enabled: True


In [12]:
try:
    config = ConfigurationManager()

    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)

except Exception as e:
    raise e

[2025-11-06 17:35:36,648: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-06 17:35:36,673: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-06 17:35:36,690: INFO: common: created directory at: artifacts]
[2025-11-06 17:35:36,692: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-11-06 17:35:36,692: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-11-06 17:35:36,700: INFO: common: created directory at: artifacts\training]
[2025-11-06 17:35:38,156: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model recompiled successfully with new optimizer.
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\admin\anaconda3\envs\chicken\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.4951 - loss: 12.1388 [2025-11-06 17:43:44,908: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
19/19 ━━━━━━━━━━━━━━━━━━━━ 480s 24s/step - accuracy: 0.5101 - loss: 13.3427 - val_accuracy: 0.7344 - val_loss: 1.4850
Epoch 2/10
 1/19 ━━━━━━━━━━━━━━━━━━━━ 6:53 23s/step - accuracy: 0.8750 - loss: 0.5433

c:\Users\admin\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


19/19 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.8750 - loss: 0.5433 - val_accuracy: 0.6094 - val_loss: 3.5915
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 463s 24s/step - accuracy: 0.6419 - loss: 6.3812 - val_accuracy: 0.5000 - val_loss: 6.6255
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.3125 - loss: 9.2052 - val_accuracy: 0.6094 - val_loss: 18.1850
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 455s 24s/step - accuracy: 0.7365 - loss: 4.5431 - val_accuracy: 0.8438 - val_loss: 2.1715
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.6250 - loss: 2.7456 - val_accuracy: 0.4688 - val_loss: 8.9051
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.7177 - loss: 3.8435 [2025-11-06 18:08:52,827: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.sav